In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Panzhihua,0,CN,1587079437,30,26.59,101.71,68.59,0.47
1,Te Anau,85,NZ,1587079437,71,-45.42,167.72,55.99,4.54
2,Ranong,99,TH,1587079437,79,9.97,98.63,80.24,3.09
3,Ibirama,0,BR,1587079438,82,-27.06,-49.52,60.01,1.70
4,Albany,40,US,1587079438,30,42.60,-73.97,41.00,17.22
...,...,...,...,...,...,...,...,...,...
592,Pisco,11,PE,1587079298,30,-13.70,-76.22,75.99,19.46
593,Kouango,38,CF,1587079517,49,4.99,19.96,76.44,1.43
594,Gillingham,0,GB,1587079518,81,51.39,0.55,53.60,8.05
595,High Level,75,CA,1587079276,37,58.52,-117.14,55.40,9.17


In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

humidity = cities_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# vacay_df = cities_df.loc[cities_df["Max Temp"]<80, ["Max Temp"]>70, ["Wind Speed"]<10]
temp = (cities_df["Max Temp"]<80) & (cities_df["Max Temp"]>70)
wind = (cities_df["Wind Speed"]<10)
cloud = (cities_df["Cloudiness"]==0)

cities_df = cities_df[temp & wind & cloud]
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
181,Morondava,0,MG,1587079463,72,-20.28,44.28,77.23,6.60
331,Mossamedes,0,AO,1587079469,61,-15.20,12.15,75.20,5.41
375,Rāzole,0,IN,1587079487,87,16.48,81.83,79.11,4.97
431,Rāzampeta,0,IN,1587079495,80,14.18,79.17,74.73,4.99
584,Emerald,0,AU,1587079516,69,-23.53,148.17,77.00,3.36


In [6]:
cities_df["Hotel Name"] = ""
cities_df

/Users/mansi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
181,Morondava,0,MG,1587079463,72,-20.28,44.28,77.23,6.60,
331,Mossamedes,0,AO,1587079469,61,-15.20,12.15,75.20,5.41,
375,Rāzole,0,IN,1587079487,87,16.48,81.83,79.11,4.97,
431,Rāzampeta,0,IN,1587079495,80,14.18,79.17,74.73,4.99,
584,Emerald,0,AU,1587079516,69,-23.53,148.17,77.00,3.36,


In [7]:
hotel_df = pd.DataFrame(cities_df, columns = ["City", "Country", "Lat", "Lng", "Hotel Name"])
hotel_df

# cities_df

,City,Country,Lat,Lng,Hotel Name
181,Morondava,MG,-20.28,44.28,
331,Mossamedes,AO,-15.20,12.15,
375,Rāzole,IN,16.48,81.83,
431,Rāzampeta,IN,14.18,79.17,
584,Emerald,AU,-23.53,148.17,


In [8]:
# cities_df.loc["Lat"]

In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f" {lat},{lng}"
    
    response = requests.get(base_url, params=params).json()
#     hotel_location = response.json
#     print(hotel_location)
    results = response['results']
    
    
    try:
#         print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [10]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
181,Morondava,MG,-20.28,44.28,La Case Bambou
331,Mossamedes,AO,-15.20,12.15,Chik-Chik Namibe
375,Rāzole,IN,16.48,81.83,Hotel Somisetty Landmark Konaseema
431,Rāzampeta,IN,14.18,79.17,Maruthi Prasanna
584,Emerald,AU,-23.53,148.17,The Irish Village


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# fig.add_layer(info_box_template)
# fig
figs = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# fig.add_layer(hotel_layer)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))